In [2]:
import pandas as pd
import json
import re
from tqdm import tqdm
from projects.mixRAG.WikidataLoader import WikiDataLoader

COMPMIX_IR_PATH = "CompMix-IR/kb_wikipedia_mixed_rd.json"
COMPMIX_PATH = "CompMix/train_set.jsonl"
dataloader = WikiDataLoader()

In [3]:

infoboxes, wikitables = dataloader.load_tables(qid="Q493078")

display(len(infoboxes))
display(len(wikitables))

1

40

In [15]:

def read_jsonl(path):
    with open(path, "r", encoding="utf-8") as f:
        qa = f.readlines()
    qa = [json.loads(line) for line in qa]
    return pd.DataFrame(qa)


df = read_jsonl(COMPMIX_PATH)
df.head()



,question_id,question,domain,entities,answers,answer_src,answer_text,convmix_question_id
0,3642,What is the genre of the tv series High Seas?,tvseries,"[{'id': 'Q186424', 'label': 'detective fiction'}]","[{'id': 'Q59591953', 'label': 'High Seas'}]",kb,detective fiction,1236-0
1,2082,What year was M.O.V.E first formed?,music,"[{'id': 'Q285547', 'label': 'Move'}]","[{'id': '1997-01-01T00:00:00Z', 'label': '1997'}]",infobox,1997,694-1
2,4693,What year was the inception of the soccer club...,soccer,"[{'id': 'Q18656', 'label': 'Manchester United ...","[{'id': '1878-01-01T00:00:00Z', 'label': '1878'}]",kb,1878,1588-4
3,2149,Who were the creators of the situation comedy ...,tvseries,"[{'id': 'Q615944', 'label': 'The Mary Tyler Mo...","[{'id': 'Q95125', 'label': 'James L. Brooks'},...",kb,James L. Brooks and Allan Burns,716-2
4,2212,Which country did the TV series Coupling origi...,tvseries,"[{'id': 'Q1137587', 'label': 'Coupling'}]","[{'id': 'Q145', 'label': 'United Kingdom'}]",infobox,United Kingdom,737-0


### 下载table数据
使用wikidataloader下载对应qid的text、table、infobox


In [3]:
import re
import json

table_path = "CompMix/tableqa_sample.csv"
df = pd.read_csv(table_path)

def convert_to_json(s):
    # 先处理字典的单引号，但保留词中的撇号
    # 使用正则表达式匹配字典的键值对中的单引号
    s = re.sub(r"'(id|label)':", r'"\1":', s)
    # 处理值中的单引号，但排除所有格撇号
    s = re.sub(r":\s*'([^']*)'", r': "\1"', s)
    return s


def get_entities(df):
    entities = []
    for i,row in df.iterrows():
        try:
            entity_str = convert_to_json(row['entities'])
            ej = json.loads(entity_str)
            for entity in ej:
                if entity not in entities:
                    entities.append(entity)
        except Exception as error:
            print(f"row: {i},entities: {row['entities']}")
            print(f"error: {error}")
    return entities

entities = get_entities(df)
# entities[:5]

for entity in tqdm(entities):
    qid = entity['id']
    print(f"qid: {qid}")
    dataloader.load_wikidata(qid)

,question_id,question,domain,entities,answers,answer_src,answer_text,convmix_question_id
0,4052,What is Russell Crowe's date of birth?,movies,"[{'id': 'Q129817', 'label': 'Russell Crowe'}]","[{'id': '1964-04-07T00:00:00Z', 'label': '7 Ap...",table,7 April 1964,1372-3
1,1057,Who won the Best Actor award in the Los Angele...,movies,"[{'id': 'Q47221', 'label': 'Taxi Driver'}]","[{'id': 'Q36949', 'label': 'Robert De Niro'}]",table,Robert De Niro,352-2
2,1575,Lin-Manuel Miranda played the role of who in t...,music,"[{'id': 'Q28659892', 'label': 'list of awards ...","[{'id': 'Q178903', 'label': 'Alexander Hamilto...",table,Alexander Hamilton,524-4
3,2992,which date was Emily in Paris season 1 released?,tvseries,"[{'id': 'Q64584978', 'label': 'Emily in Paris'}]","[{'id': '2020-10-02T00:00:00Z', 'label': '2 Oc...",table,"October 2, 2020",1017-0
4,4249,Winner of the 2021 MLS cup?,soccer,"[{'id': 'Q577698', 'label': 'MLS Cup'}]","[{'id': 'Q13361030', 'label': 'New York City F...",table,New York City FC,1439-0


## 解析text

In [ ]:
qid = "Q64584978"
page = pd.read_html(dataloader.get_url_from_qid(qid))
for df in page:
    display(df.head())

In [6]:
qid = "Q64584978"
infoboxes, tables = dataloader.load_tables(qid)

for table in tables:
    df = pd.DataFrame(columns=table['table_headers'], data=table['table_text'])
    display(df.head())


,Season,Episodes,Episodes.1,Originally released,Originally released.1
0,Season,Episodes,Episodes.1,Originally released,Originally released.1
1,1,10,10,"October 2, 2020","October 2, 2020"
2,2,10,10,"December 22, 2021","December 22, 2021"
3,3,10,10,"December 21, 2022","December 21, 2022"
4,4,10,5,"August 15, 2024","August 15, 2024"


,No. overall,No. in season,Title,Directed by,Written by,Original release date,Unnamed: 6
0,No. overall,No. in season,Title,Directed by,Written by,Original release date,Unnamed: 6
1,1,1,"""Emily in Paris""",Andrew Fleming,Darren Star,"October 2, 2020",NaN
2,Emily Cooper's boss Madeline prepares to trans...,Emily Cooper's boss Madeline prepares to trans...,Emily Cooper's boss Madeline prepares to trans...,Emily Cooper's boss Madeline prepares to trans...,Emily Cooper's boss Madeline prepares to trans...,Emily Cooper's boss Madeline prepares to trans...,Emily Cooper's boss Madeline prepares to trans...
3,2,2,"""Masculin Féminin""",Andrew Fleming,Darren Star,"October 2, 2020",NaN
4,Despite struggling to fit in with French offic...,Despite struggling to fit in with French offic...,Despite struggling to fit in with French offic...,Despite struggling to fit in with French offic...,Despite struggling to fit in with French offic...,Despite struggling to fit in with French offic...,Despite struggling to fit in with French offic...


,No. overall,No. in season,Title,Directed by,Written by,Original release date,Unnamed: 6
0,No. overall,No. in season,Title,Directed by,Written by,Original release date,Unnamed: 6
1,11,1,"""Voulez-Vous Coucher Avec Moi?""",Andrew Fleming,Darren Star,"December 22, 2021",NaN
2,Emily is confused after her affair with Gabrie...,Emily is confused after her affair with Gabrie...,Emily is confused after her affair with Gabrie...,Emily is confused after her affair with Gabrie...,Emily is confused after her affair with Gabrie...,Emily is confused after her affair with Gabrie...,Emily is confused after her affair with Gabrie...
3,12,2,"""Do You Know the Way to St. Tropez?""",Andrew Fleming,Grant Sloss,"December 22, 2021",NaN
4,Emily ends up staying at the Four Seasons Hote...,Emily ends up staying at the Four Seasons Hote...,Emily ends up staying at the Four Seasons Hote...,Emily ends up staying at the Four Seasons Hote...,Emily ends up staying at the Four Seasons Hote...,Emily ends up staying at the Four Seasons Hote...,Emily ends up staying at the Four Seasons Hote...


,No. overall,No. in season,Title,Directed by,Written by,Original release date,Unnamed: 6
0,No. overall,No. in season,Title,Directed by,Written by,Original release date,Unnamed: 6
1,21,1,"""I Have Two Lovers""",Andrew Fleming,Grant Sloss,"December 21, 2022",NaN
2,Emily dreams of standing at the Eiffel Tower a...,Emily dreams of standing at the Eiffel Tower a...,Emily dreams of standing at the Eiffel Tower a...,Emily dreams of standing at the Eiffel Tower a...,Emily dreams of standing at the Eiffel Tower a...,Emily dreams of standing at the Eiffel Tower a...,Emily dreams of standing at the Eiffel Tower a...
3,22,2,"""What's It All About...""",Andrew Fleming,Alison Brown,"December 21, 2022",NaN
4,"Madeline gives birth to a baby boy named Jack,...","Madeline gives birth to a baby boy named Jack,...","Madeline gives birth to a baby boy named Jack,...","Madeline gives birth to a baby boy named Jack,...","Madeline gives birth to a baby boy named Jack,...","Madeline gives birth to a baby boy named Jack,...","Madeline gives birth to a baby boy named Jack,..."


,No. overall,No. in season,Title,Directed by,Written by,Original release date,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,No. overall,No. in season,Title,Directed by,Written by,Original release date,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
1,Part 1,Part 1,Part 1,Part 1,Part 1,Part 1,Part 1,Part 1,Part 1,Part 1,Part 1,Part 1,Part 1
2,31,1,"""Break Point""",Andrew Fleming,Grant Sloss,"August 15, 2024",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Camille's brother throws shade at Emily on his...,Camille's brother throws shade at Emily on his...,Camille's brother throws shade at Emily on his...,Camille's brother throws shade at Emily on his...,Camille's brother throws shade at Emily on his...,Camille's brother throws shade at Emily on his...,Camille's brother throws shade at Emily on his...,NaN,NaN,NaN,NaN,NaN,NaN
4,32,2,"""Love on the Run""",Andrew Fleming,Alison Brown,"August 15, 2024",NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Season,Rotten Tomatoes,Metacritic
0,Season,Rotten Tomatoes,Metacritic
1,1,61% (57 reviews)[61],58 (18 reviews)[62]
2,2,61% (23 reviews)[63],64 (11 reviews)[64]
3,3,67% (18 reviews)[65],54 (10 reviews)[66]
4,4,65% (31 reviews)[67],47 (13 reviews)[68]


,Year,Award,Category,Recipient(s),Result,Ref.
0,Year,Award,Category,Recipient(s),Result,Ref.
1,2021,Art Directors Guild Awards,Excellence in Production Design for a Half Hou...,"Anne Seibel (for ""Emily in Paris"")",Nominated,[98]
2,2021,Costume Designers Guild Awards,Excellence in Contemporary Television,"Patricia Field and Marylin Fitoussi (for ""Faux...",Nominated,[99]
3,2021,Critics' Choice Television Awards,Best Supporting Actress in a Comedy Series,Ashley Park,Nominated,[100]
4,2021,Golden Globe Awards,Best Television Series – Musical or Comedy,Emily in Paris,Nominated,[101][102]


In [5]:
a = {1:2, 2:3,3:0}
b = sorted(a.items(), key=lambda x: x[1], reverse=True)
b
c = list(a.items())
c

[(1, 2), (2, 3), (3, 0)]